In [1]:
library(dplyr)
library(caret)
library(nnet)


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Loading required package: lattice
Loading required package: ggplot2
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang


In [2]:
data <- read.csv("./test_result.csv", fileEncoding = "utf-8")

In [3]:
data$cluster <- as.factor(data$cluster)

In [4]:
str(data)

'data.frame':	322 obs. of  14 variables:
 $ 영화명    : Factor w/ 57 levels "1987","감기",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ 누적매출액: num  5.82e+10 5.82e+10 5.82e+10 5.82e+10 5.82e+10 ...
 $ 누적관객수: int  7232387 7232387 7232387 7232387 7232387 7232387 7232387 7232387 7232387 7232387 ...
 $ 스크린수  : int  1299 1299 1299 1299 1299 1299 1299 1299 1299 1299 ...
 $ 상영횟수  : int  138226 138226 138226 138226 138226 138226 138226 138226 138226 138226 ...
 $ 개봉일    : Factor w/ 55 levels "2005-07-29","2006-09-28",..: 47 47 47 47 47 47 47 47 47 47 ...
 $ 배급사    : Factor w/ 12 levels "(주)넥스트엔터테인먼트월드(NEW)",..: 4 4 4 4 4 4 4 4 4 4 ...
 $ 등급      : Factor w/ 5 levels "12세이상관람가",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ 장르      : Factor w/ 34 levels "SF,액션","드라마",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ 감독      : Factor w/ 41 levels "강우석","강형철",..: 34 34 34 34 34 34 34 34 34 34 ...
 $ 배우      : Factor w/ 142 levels "T.O.P","강동원",..: 22 137 82 31 53 103 22 137 82 31 ...
 $ 제작사    : Factor w/ 51 levels "(유)내부자들 문화전문회사",..: 34 34 34 34 34 

In [5]:
sdata <- data

In [6]:
data <- subset(data, select = c(스크린수, 상영횟수, 감독, 배우, cluster, 감독_배우))

In [7]:
table(data$cluster)


  1   2   3 
 71  97 154 

In [8]:
data

스크린수,상영횟수,감독,배우,cluster,감독_배우
1299,138226,장준환,김윤석,3,하정우
1299,138226,장준환,하정우,3,하정우
1299,138226,장준환,유해진,3,하정우
1299,138226,장준환,김태리,3,하정우
1299,138226,장준환,박희순,3,하정우
1299,138226,장준환,이희준,3,하정우
1299,138226,장준환,김윤석,3,유해진
1299,138226,장준환,하정우,3,유해진
1299,138226,장준환,유해진,3,유해진
1299,138226,장준환,김태리,3,유해진


In [9]:
test <- read.csv("./test_result2.csv", fileEncoding = "utf-8")

In [10]:
test

영화명,누적매출액,누적관객수,스크린수,상영횟수,개봉일,배급사,등급,장르,감독,배우,제작사,cluster,감독_배우
1987,58169776145,7232387,1299,138226,2017-12-27,(주)씨제이이엔엠,15세이상관람가,드라마,장준환,김윤석,(주)우정필름,3,하정우
1987,58169776145,7232387,1299,138226,2017-12-27,(주)씨제이이엔엠,15세이상관람가,드라마,장준환,하정우,(주)우정필름,3,하정우
1987,58169776145,7232387,1299,138226,2017-12-27,(주)씨제이이엔엠,15세이상관람가,드라마,장준환,김윤석,(주)우정필름,3,유해진
1987,58169776145,7232387,1299,138226,2017-12-27,(주)씨제이이엔엠,15세이상관람가,드라마,장준환,하정우,(주)우정필름,3,유해진
1987,58169776145,7232387,1299,138226,2017-12-27,(주)씨제이이엔엠,15세이상관람가,드라마,장준환,김윤석,(주)우정필름,3,김윤석
1987,58169776145,7232387,1299,138226,2017-12-27,(주)씨제이이엔엠,15세이상관람가,드라마,장준환,하정우,(주)우정필름,3,김윤석
감기,21774884000,3117859,806,55046,2013-08-14,아이러브시네마,15세이상관람가,"드라마,어드벤처,액션",김성수,장혁,"아이러브시네마,(주)아이필름홀딩스,(주)씨제이이엔엠",1,유해진
감기,21774884000,3117859,806,55046,2013-08-14,아이러브시네마,15세이상관람가,"드라마,어드벤처,액션",김성수,수애,"아이러브시네마,(주)아이필름홀딩스,(주)씨제이이엔엠",1,유해진
강철비,35490346466,4452850,1426,80359,2017-12-14,(주)넥스트엔터테인먼트월드(NEW),15세이상관람가,"액션,드라마",양우석,정우성,"(주)와이웍스엔터테인먼트,(주)모팩",2,이경영
강철비,35490346466,4452850,1426,80359,2017-12-14,(주)넥스트엔터테인먼트월드(NEW),15세이상관람가,"액션,드라마",양우석,곽도원,"(주)와이웍스엔터테인먼트,(주)모팩",2,이경영


In [11]:
table(test$cluster)


 1  2  3 
38 62 70 

In [12]:
test <- subset(test, select = c(스크린수, 상영횟수, 감독, 배우, cluster, 감독_배우))

In [13]:
test$cluster <- as.factor(test$cluster)

In [14]:
test

스크린수,상영횟수,감독,배우,cluster,감독_배우
1299,138226,장준환,김윤석,3,하정우
1299,138226,장준환,하정우,3,하정우
1299,138226,장준환,김윤석,3,유해진
1299,138226,장준환,하정우,3,유해진
1299,138226,장준환,김윤석,3,김윤석
1299,138226,장준환,하정우,3,김윤석
806,55046,김성수,장혁,1,유해진
806,55046,김성수,수애,1,유해진
1426,80359,양우석,정우성,2,이경영
1426,80359,양우석,곽도원,2,이경영


In [15]:
data <- arrange(data, cluster)

In [16]:
data

스크린수,상영횟수,감독,배우,cluster,감독_배우
806,55046,김성수,장혁,1,유해진
806,55046,김성수,수애,1,유해진
806,55046,김성수,박민하,1,유해진
707,74157,강우석,설경구,1,유해진
707,74157,강우석,정재영,1,유해진
432,69946,이연우,김윤석,1,김윤석
432,69946,이연우,정경호,1,김윤석
432,69946,이연우,견미리,1,김윤석
432,69946,이연우,선우선,1,김윤석
956,95437,최성현,이병헌,1,이병헌


In [17]:
samp <- createDataPartition(data$cluster, p=0.7, list=F)
# 학습용 데이터 셋
data.tr <- data[samp,]

# 검증용 데이터셋
data.test <- data[-samp,]

# 문제 및 정답 만들기
x <- subset(data.test, select = -cluster)
y <- data.test$cluster

# Model 만들기
model.nnet <- nnet(cluster ~ ., data=data.tr, size = 5) 

pred <- predict(model.nnet, x, type = "class")

mean(pred == y)
table(pred, y)

# weights:  983
initial  value 276.319345 
final  value 236.843063 
converged


[1] 0.4791667

    y
pred  1  2  3
   3 21 29 46

In [18]:
samp <- createDataPartition(data$cluster, p=0.7, list=F)
# 학습용 데이터 셋
data.tr <- data[samp,c(5,3,4,6)]

# 검증용 데이터셋
data.test <- data[-samp,c(5,3,4,6)]

# 문제 및 정답 만들기
x <- subset(data.test, select = -cluster)
y <- data.test$cluster

# Model 만들기
model.nnet <- nnet(cluster ~ ., data=data.tr, size = 5) 

pred <- predict(model.nnet, x, type = "class")

mean(pred == y)
table(pred, y)

# weights:  973
initial  value 259.066311 
iter  10 value 19.025368
iter  20 value 2.880195
iter  30 value 2.775608
iter  40 value 2.773067
iter  50 value 2.772752
iter  60 value 2.772665
iter  70 value 2.772643
iter  80 value 2.772607
final  value 2.772602 
converged


[1] 0.8958333

    y
pred  1  2  3
   1 17  1  3
   2  3 27  1
   3  1  1 42

In [19]:
samp <- createDataPartition(test$cluster, p=0.7, list=F)
# 학습용 데이터 셋
data.tr <- test[samp,c(5,3,4,6)]

# 검증용 데이터셋
data.test <- test[-samp,c(5,3,4,6)]

# 문제 및 정답 만들기
x <- subset(data.test, select = -cluster)
y <- data.test$cluster

# Model 만들기
model.nnet <- nnet(cluster ~ ., data=data.tr, size = 5) 

pred <- predict(model.nnet, x, type = "class")

mean(pred == y)
table(pred, y)

# weights:  578
initial  value 134.449261 
iter  10 value 5.268872
iter  20 value 1.397444
iter  30 value 1.387075
iter  40 value 1.386644
iter  50 value 1.386387
iter  60 value 1.386330
iter  70 value 1.386295
final  value 1.386295 
converged


[1] 0.78

    y
pred  1  2  3
   1  3  1  0
   2  6 16  1
   3  2  1 20

In [20]:
pred_res <- c()

for (i in 1:100) {
    samp <- createDataPartition(data$cluster, p=0.7, list=F)
    # 학습용 데이터 셋
    data.tr <- data[samp,c(5,3,4,6)]

    # 검증용 데이터셋
    data.test <- data[-samp,c(5,3,4,6)]

    # 문제 및 정답 만들기
    x <- subset(data.test, select = -cluster)
    y <- data.test$cluster

    # Model 만들기
    model.nnet <- nnet(cluster ~ ., data=data.tr, size = 5) 

    pred <- predict(model.nnet, x, type = "class")

    mean(pred == y)

    pred_res <- append(pred_res,mean(pred == y))
}
mean(pred_res)

# weights:  973
initial  value 236.540825 
iter  10 value 8.834286
iter  20 value 2.800249
iter  30 value 2.774006
iter  40 value 2.773100
iter  50 value 2.772648
iter  60 value 2.772603
iter  70 value 2.772601
final  value 2.772600 
converged
# weights:  973
initial  value 269.796708 
iter  10 value 71.996072
iter  20 value 0.431569
iter  30 value 0.003357
iter  40 value 0.000586
final  value 0.000094 
converged
# weights:  973
initial  value 309.136267 
iter  10 value 24.313463
iter  20 value 2.949203
iter  30 value 2.779819
iter  40 value 2.773245
iter  50 value 2.772614
iter  60 value 2.772600
iter  70 value 2.772595
iter  80 value 2.772593
final  value 2.772593 
converged
# weights:  973
initial  value 253.912945 
iter  10 value 54.487670
iter  20 value 2.466307
iter  30 value 1.411774
iter  40 value 1.391794
iter  50 value 1.386906
iter  60 value 1.386527
iter  70 value 1.386496
iter  80 value 1.386375
iter  90 value 1.386369
iter 100 value 1.386356
final  value 1.386356 
stopped

initial  value 293.293463 
iter  10 value 65.833251
iter  20 value 3.449987
iter  30 value 2.786319
iter  40 value 2.774578
iter  50 value 2.773188
iter  60 value 2.772793
iter  70 value 2.772644
iter  80 value 2.772603
iter  90 value 2.772597
final  value 2.772597 
converged
# weights:  973
initial  value 281.353404 
iter  10 value 21.132428
iter  20 value 2.942493
iter  30 value 2.778524
iter  40 value 2.774181
iter  50 value 2.773260
iter  60 value 2.772774
iter  70 value 2.772699
iter  80 value 2.772613
final  value 2.772610 
converged
# weights:  973
initial  value 255.830948 
iter  10 value 10.511145
iter  20 value 1.422557
iter  30 value 1.387764
iter  40 value 1.387060
iter  50 value 1.386627
iter  60 value 1.386321
iter  70 value 1.386310
iter  80 value 1.386305
iter  90 value 1.386301
iter 100 value 1.386299
final  value 1.386299 
stopped after 100 iterations
# weights:  973
initial  value 272.956754 
iter  10 value 62.360403
iter  20 value 0.760972
iter  30 value 0.013320
it

initial  value 273.230036 
iter  10 value 39.358102
iter  20 value 1.427163
iter  30 value 1.391668
iter  40 value 1.387134
iter  50 value 1.386607
iter  60 value 1.386523
iter  70 value 1.386444
iter  80 value 1.386402
iter  90 value 1.386376
iter 100 value 1.386338
final  value 1.386338 
stopped after 100 iterations
# weights:  973
initial  value 249.509196 
iter  10 value 54.873739
iter  20 value 3.702123
iter  30 value 2.809930
iter  40 value 2.774570
iter  50 value 2.773721
iter  60 value 2.773002
iter  70 value 2.772807
iter  80 value 2.772713
iter  90 value 2.772688
iter 100 value 2.772663
final  value 2.772663 
stopped after 100 iterations
# weights:  973
initial  value 300.482504 
iter  10 value 92.561926
iter  20 value 7.761374
iter  30 value 2.828002
iter  40 value 2.776213
iter  50 value 2.773839
iter  60 value 2.772694
iter  70 value 2.772643
iter  80 value 2.772628
iter  90 value 2.772595
iter 100 value 2.772591
final  value 2.772591 
stopped after 100 iterations
# weight

initial  value 346.141569 
iter  10 value 138.889897
iter  20 value 3.189056
iter  30 value 2.789780
iter  40 value 2.772756
iter  50 value 2.772699
iter  60 value 2.772643
final  value 2.772633 
converged
# weights:  973
initial  value 242.754705 
iter  10 value 7.591761
iter  20 value 1.411881
iter  30 value 1.387825
iter  40 value 1.386618
iter  50 value 1.386333
iter  60 value 1.386324
iter  70 value 1.386318
iter  80 value 1.386302
iter  90 value 1.386300
final  value 1.386299 
converged
# weights:  973
initial  value 269.275855 
iter  10 value 45.784470
iter  20 value 11.144638
iter  30 value 3.260796
iter  40 value 2.806578
iter  50 value 2.778435
iter  60 value 2.774410
iter  70 value 2.772997
iter  80 value 2.772799
iter  90 value 2.772739
iter 100 value 2.772698
final  value 2.772698 
stopped after 100 iterations
# weights:  973
initial  value 284.780993 
iter  10 value 64.572705
iter  20 value 4.910569
iter  30 value 2.800641
iter  40 value 2.774116
iter  50 value 2.773079
i

[1] 0.9020833

In [21]:
pred_res <- c()

for (i in 1:100) {
    samp <- createDataPartition(data$cluster, p=0.7, list=F)
    # 학습용 데이터 셋
    data.tr <- data[samp,c(5,3,4)]

    # 검증용 데이터셋
    data.test <- data[-samp,c(5,3,4)]

    # 문제 및 정답 만들기
    x <- subset(data.test, select = -cluster)
    y <- data.test$cluster

    # Model 만들기
    model.nnet <- nnet(cluster ~ ., data=data.tr, size = 5) 

    pred <- predict(model.nnet, x, type = "class")

    mean(pred == y)

    pred_res <- append(pred_res,mean(pred == y))
}
mean(pred_res)

# weights:  928
initial  value 261.792995 
iter  10 value 25.898193
iter  20 value 1.494668
iter  30 value 1.389606
iter  40 value 1.387469
iter  50 value 1.386399
iter  60 value 1.386363
iter  70 value 1.386338
iter  80 value 1.386332
iter  90 value 1.386307
iter 100 value 1.386297
final  value 1.386297 
stopped after 100 iterations
# weights:  928
initial  value 283.723374 
iter  10 value 36.233257
iter  20 value 4.350046
iter  30 value 3.306171
iter  40 value 3.296661
iter  50 value 3.295963
iter  60 value 3.295891
iter  70 value 3.295868
iter  80 value 3.295864
final  value 3.295851 
converged
# weights:  928
initial  value 365.472973 
iter  10 value 49.445345
iter  20 value 11.674464
iter  30 value 3.626221
iter  40 value 3.308365
iter  50 value 3.296945
iter  60 value 3.296056
iter  70 value 3.295851
iter  80 value 3.295844
iter  90 value 3.295841
iter 100 value 3.295839
final  value 3.295839 
stopped after 100 iterations
# weights:  928
initial  value 304.847240 
iter  10 value 

initial  value 246.345988 
iter  10 value 16.854131
iter  20 value 1.957965
iter  30 value 1.911873
iter  40 value 1.909844
iter  50 value 1.909624
iter  60 value 1.909575
iter  70 value 1.909569
iter  70 value 1.909569
iter  70 value 1.909569
final  value 1.909569 
converged
# weights:  928
initial  value 244.498698 
iter  10 value 26.956333
iter  20 value 3.685953
iter  30 value 3.297306
iter  40 value 3.296079
iter  50 value 3.295925
iter  60 value 3.295880
final  value 3.295854 
converged
# weights:  928
initial  value 289.925555 
iter  10 value 80.223570
iter  20 value 8.654308
iter  30 value 5.236876
iter  40 value 4.694869
iter  50 value 4.684769
iter  60 value 4.683212
iter  70 value 4.682535
final  value 4.682175 
converged
# weights:  928
initial  value 240.047624 
iter  10 value 67.120260
iter  20 value 3.142196
iter  30 value 2.785217
iter  40 value 2.774469
iter  50 value 2.773023
iter  60 value 2.772876
iter  70 value 2.772640
iter  80 value 2.772594
final  value 2.772592

initial  value 275.766892 
iter  10 value 54.661470
iter  20 value 7.374837
iter  30 value 2.806118
iter  40 value 2.773015
iter  50 value 2.772786
iter  60 value 2.772618
iter  70 value 2.772602
iter  80 value 2.772592
final  value 2.772591 
converged
# weights:  928
initial  value 308.959518 
iter  10 value 41.330489
iter  20 value 3.788272
iter  30 value 3.300672
iter  40 value 3.296956
iter  50 value 3.295859
iter  60 value 3.295841
final  value 3.295841 
converged
# weights:  928
initial  value 249.095544 
iter  10 value 26.398116
iter  20 value 2.947627
iter  30 value 2.776425
iter  40 value 2.773143
iter  50 value 2.772760
iter  60 value 2.772629
iter  70 value 2.772609
iter  80 value 2.772601
final  value 2.772601 
converged
# weights:  928
initial  value 247.841112 
iter  10 value 11.206122
iter  20 value 2.869864
iter  30 value 2.775280
iter  40 value 2.773116
iter  50 value 2.772911
iter  60 value 2.772706
iter  70 value 2.772612
iter  80 value 2.772606
iter  90 value 2.7725

initial  value 277.831256 
iter  10 value 7.678556
iter  20 value 1.482403
iter  30 value 1.389079
iter  40 value 1.386568
iter  50 value 1.386419
iter  60 value 1.386387
iter  70 value 1.386374
iter  80 value 1.386368
iter  90 value 1.386337
iter 100 value 1.386314
final  value 1.386314 
stopped after 100 iterations
# weights:  928
initial  value 244.960809 
iter  10 value 33.917984
iter  20 value 1.476741
iter  30 value 1.389234
iter  40 value 1.386641
iter  50 value 1.386387
iter  60 value 1.386316
iter  70 value 1.386313
iter  80 value 1.386299
final  value 1.386299 
converged
# weights:  928
initial  value 296.836370 
iter  10 value 31.606600
iter  20 value 1.816269
iter  30 value 1.387249
iter  40 value 1.386369
iter  50 value 1.386336
iter  60 value 1.386299
final  value 1.386298 
converged
# weights:  928
initial  value 278.124043 
iter  10 value 25.035490
iter  20 value 3.918982
iter  30 value 3.822983
iter  40 value 3.819403
iter  50 value 3.819286
iter  60 value 3.819140
ite

[1] 0.8772917

In [22]:
pred <- predict(model.nnet, data.frame(감독="강우석",배우="유해진",감독_배우="유해진"), type = "class")

In [23]:
pred

[1] "1"

In [25]:
sdata

영화명,누적매출액,누적관객수,스크린수,상영횟수,개봉일,배급사,등급,장르,감독,배우,제작사,cluster,감독_배우
1987,58169776145,7232387,1299,138226,2017-12-27,(주)씨제이이엔엠,15세이상관람가,드라마,장준환,김윤석,(주)우정필름,3,하정우
1987,58169776145,7232387,1299,138226,2017-12-27,(주)씨제이이엔엠,15세이상관람가,드라마,장준환,하정우,(주)우정필름,3,하정우
1987,58169776145,7232387,1299,138226,2017-12-27,(주)씨제이이엔엠,15세이상관람가,드라마,장준환,유해진,(주)우정필름,3,하정우
1987,58169776145,7232387,1299,138226,2017-12-27,(주)씨제이이엔엠,15세이상관람가,드라마,장준환,김태리,(주)우정필름,3,하정우
1987,58169776145,7232387,1299,138226,2017-12-27,(주)씨제이이엔엠,15세이상관람가,드라마,장준환,박희순,(주)우정필름,3,하정우
1987,58169776145,7232387,1299,138226,2017-12-27,(주)씨제이이엔엠,15세이상관람가,드라마,장준환,이희준,(주)우정필름,3,하정우
1987,58169776145,7232387,1299,138226,2017-12-27,(주)씨제이이엔엠,15세이상관람가,드라마,장준환,김윤석,(주)우정필름,3,유해진
1987,58169776145,7232387,1299,138226,2017-12-27,(주)씨제이이엔엠,15세이상관람가,드라마,장준환,하정우,(주)우정필름,3,유해진
1987,58169776145,7232387,1299,138226,2017-12-27,(주)씨제이이엔엠,15세이상관람가,드라마,장준환,유해진,(주)우정필름,3,유해진
1987,58169776145,7232387,1299,138226,2017-12-27,(주)씨제이이엔엠,15세이상관람가,드라마,장준환,김태리,(주)우정필름,3,유해진


In [29]:
length(as.character(unique(sdata$영화명)))

[1] 57

In [30]:
as.character(unique(sdata$영화명))[1]

[1] "1987"

In [31]:
round(5*0.7)

[1] 4

In [41]:
samp <- c()
for (i in 1:length(as.character(unique(sdata$영화명)))) {
    temp <- c()
    for (j in 1:nrow(sdata)) {
        if (sdata$영화명[j] == as.character(unique(sdata$영화명))[i]) {
            temp <- append(temp, j)
        }
    }
    samp <- append(samp, sample(temp, round(length(temp)*0.7)))
}

In [42]:

# 학습용 데이터 셋
data.tr <- data[samp,c(5,3,4,6)]

# 검증용 데이터셋
data.test <- data[-samp,c(5,3,4,6)]

# 문제 및 정답 만들기
x <- subset(data.test, select = -cluster)
y <- data.test$cluster

# Model 만들기
model.nnet <- nnet(cluster ~ ., data=data.tr, size = 5) 

pred <- predict(model.nnet, x, type = "class")

mean(pred == y)
table(pred, y)

# weights:  973
initial  value 243.655349 
iter  10 value 115.565039
iter  20 value 3.270509
iter  30 value 1.416195
iter  40 value 1.389345
iter  50 value 1.387351
iter  60 value 1.386833
iter  70 value 1.386447
iter  80 value 1.386330
iter  90 value 1.386308
iter 100 value 1.386300
final  value 1.386300 
stopped after 100 iterations


[1] 0.85

    y
pred  1  2  3
   1 18  1  2
   2  0 30 10
   3  2  0 37

In [43]:
pred_res <- c()

for (i in 1:100) {
    samp <- c()
    for (i in 1:length(as.character(unique(sdata$영화명)))) {
        temp <- c()
        for (j in 1:nrow(sdata)) {
            if (sdata$영화명[j] == as.character(unique(sdata$영화명))[i]) {
                temp <- append(temp, j)
            }
        }
        samp <- append(samp, sample(temp, round(length(temp)*0.7)))
    }
    # 학습용 데이터 셋
    data.tr <- data[samp,c(5,3,4)]

    # 검증용 데이터셋
    data.test <- data[-samp,c(5,3,4)]

    # 문제 및 정답 만들기
    x <- subset(data.test, select = -cluster)
    y <- data.test$cluster

    # Model 만들기
    model.nnet <- nnet(cluster ~ ., data=data.tr, size = 5) 

    pred <- predict(model.nnet, x, type = "class")

    mean(pred == y)

    pred_res <- append(pred_res,mean(pred == y))
}
mean(pred_res)

# weights:  928
initial  value 264.151495 
iter  10 value 35.060054
iter  20 value 1.618642
iter  30 value 1.398787
iter  40 value 1.389939
iter  50 value 1.386867
iter  60 value 1.386522
iter  70 value 1.386419
iter  80 value 1.386316
iter  90 value 1.386311
final  value 1.386310 
converged
# weights:  928
initial  value 265.502022 
iter  10 value 13.480886
iter  20 value 1.411256
iter  30 value 1.386780
iter  40 value 1.386538
iter  50 value 1.386299
iter  60 value 1.386295
iter  60 value 1.386295
iter  60 value 1.386295
final  value 1.386295 
converged
# weights:  928
initial  value 248.847401 
iter  10 value 24.778566
iter  20 value 5.888164
iter  30 value 1.428742
iter  40 value 1.391359
iter  50 value 1.387163
iter  60 value 1.386658
iter  70 value 1.386458
iter  80 value 1.386411
iter  90 value 1.386373
iter 100 value 1.386346
final  value 1.386346 
stopped after 100 iterations
# weights:  928
initial  value 278.786826 
iter  10 value 115.107208
iter  20 value 15.551963
iter  30

initial  value 270.611211 
iter  10 value 29.300012
iter  20 value 3.305177
iter  30 value 3.297205
iter  40 value 3.296125
iter  50 value 3.296004
iter  60 value 3.295872
final  value 3.295868 
converged
# weights:  928
initial  value 252.638926 
iter  10 value 24.904358
iter  20 value 2.662312
iter  30 value 1.923303
iter  40 value 1.913543
iter  50 value 1.910120
iter  60 value 1.909901
iter  70 value 1.909721
iter  80 value 1.909625
iter  90 value 1.909583
iter 100 value 1.909580
final  value 1.909580 
stopped after 100 iterations
# weights:  928
initial  value 239.792409 
iter  10 value 28.407795
iter  20 value 1.474269
iter  30 value 1.390902
iter  40 value 1.386829
iter  50 value 1.386325
iter  60 value 1.386304
iter  70 value 1.386299
iter  80 value 1.386296
final  value 1.386295 
converged
# weights:  928
initial  value 388.438811 
iter  10 value 116.984859
iter  20 value 6.770274
iter  30 value 1.980286
iter  40 value 1.913352
iter  50 value 1.910738
iter  60 value 1.910316
i

initial  value 246.825960 
iter  10 value 64.620493
iter  20 value 10.921804
iter  30 value 5.619238
iter  40 value 2.077798
iter  50 value 1.922682
iter  60 value 1.913064
iter  70 value 1.910219
iter  80 value 1.910127
iter  90 value 1.909951
iter 100 value 1.909762
final  value 1.909762 
stopped after 100 iterations
# weights:  928
initial  value 265.107894 
iter  10 value 20.032254
iter  20 value 3.362503
iter  30 value 3.301652
iter  40 value 3.296302
iter  50 value 3.295931
iter  60 value 3.295881
iter  70 value 3.295864
iter  80 value 3.295854
iter  90 value 3.295848
iter 100 value 3.295846
final  value 3.295846 
stopped after 100 iterations
# weights:  928
initial  value 337.310287 
iter  10 value 100.691901
iter  20 value 16.346618
iter  30 value 0.047061
iter  40 value 0.001184
final  value 0.000085 
converged
# weights:  928
initial  value 249.528023 
iter  10 value 20.775109
iter  20 value 2.874542
iter  30 value 2.779035
iter  40 value 2.773635
iter  50 value 2.772649
iter

initial  value 370.582903 
iter  10 value 62.627777
iter  20 value 6.694334
iter  30 value 4.028956
iter  40 value 3.314444
iter  50 value 3.299543
iter  60 value 3.297407
iter  70 value 3.296600
iter  80 value 3.296100
iter  90 value 3.295914
iter 100 value 3.295880
final  value 3.295880 
stopped after 100 iterations
# weights:  928
initial  value 258.492154 
iter  10 value 14.853752
iter  20 value 1.428494
iter  30 value 1.390973
iter  40 value 1.387994
iter  50 value 1.387433
iter  60 value 1.386586
iter  70 value 1.386547
iter  80 value 1.386393
iter  90 value 1.386384
iter 100 value 1.386370
final  value 1.386370 
stopped after 100 iterations
# weights:  928
initial  value 271.955983 
iter  10 value 37.908630
iter  20 value 6.445263
iter  30 value 5.204324
iter  40 value 3.314618
iter  50 value 3.298965
iter  60 value 3.296708
iter  70 value 3.296303
iter  80 value 3.296217
iter  90 value 3.296014
iter 100 value 3.295920
final  value 3.295920 
stopped after 100 iterations
# weight

[1] 0.8817